In [ ]:
#pip install pyspark

In [ ]:
#sudo su 
#apt-get update
#apt-get install openjdk-8-jdk-headless -qq > /dev/null 
#wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
#tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [24]:
#Importing libraries
from pyspark.sql import SparkSession, SQLContext
import pyspark

import os

import pandas as pd

In [13]:
#Setting environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.1-bin-hadoop3.2"

In [14]:
#Creating Spark session
spark = SparkSession \
    .builder \
    .appName("PySpark App") \
    .config("spark.jars", "jar_files/postgresql-42.3.2.jar") \
    .getOrCreate()
spark

In [15]:
#Creating Spark and SQL context
sparkcontext = pyspark.SparkContext.getOrCreate()
sqlcontext = SQLContext(sparkcontext)

In [20]:
#Setting database parameters
postgres_uri = "jdbc:postgresql://depgdb.crhso94tou3n.eu-west-2.rds.amazonaws.com:5432/claragoebel21"
user = "claragoebel21"
password = "qwerty123"

In [27]:
#Importing data
df = pd.read_csv("output_data/data_vf.csv", index_col = 0)

In [29]:
#Defining functions to read from or write to database
def read_db(db_table):
    try:
        df_spark = spark.read \
        .format("jdbc") \
        .option("url", postgres_uri) \
        .option("dbtable", db_table) \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "org.postgresql.Driver") \
        .load()
        return df_spark
    except Exception as ex:
        return ex

def write_to_db(df_spark, db_table):
    try:
        df_spark.write \
        .mode("append") \
        .format("jdbc") \
        .option("url", postgres_uri) \
        .option("dbtable", db_table) \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "org.postgresql.Driver") \
        .save()
    except Exception as ex:
        return ex

In [75]:
#Defining function to clear table in database
def truncate_db_table(df_spark, db_table):
    df_spark.write \
    .mode("overwrite") \
    .format("jdbc") \
    .option("cascadeTruncate", True) \
    .option("url", postgres_uri) \
    .option("dbtable", db_table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .save()

In [168]:
#Writing to database
df_pandas = pd.DataFrame(data = df["type"].unique(), columns = ["name"])
df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.types")

In [169]:
#Writing to database
df_pandas = pd.DataFrame(data = df["source"].unique(), columns = ["name"])
df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.sources")

In [170]:
#Writing to database
df_pandas = df[["postcode",
                "lat",
                "long",
                "pop_density",
                "public_transport_accessibility",
                "mean_income"]].drop_duplicates()
df_pandas.columns = ["postcode",
                     "latitude",
                     "longitude",
                     "population_density",
                     "pt_accessibility",
                     "mean_income"]
df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.postcodes")

In [171]:
#Writing to database
types = ['ages_0_15', 'ages_16_29', 'ages_30_44', 'ages_45_65', 'ages_66_plus']
df_pandas = pd.DataFrame(data = types, columns = ["name"])
df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.pcd_age_bins")

In [172]:
#Writing to database
types = ['couple_children_dependent', 'couple_children_nondependent', 'lone_parent', 'single', 'other_multi']
df_pandas = pd.DataFrame(data = types, columns = ["name"])
df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.pcd_household_types")

In [173]:
#Writing to database
types = ['owned_outright', 'owned_mortgage', 'rented_social', 'rented_private']
df_pandas = pd.DataFrame(data = types, columns = ["name"])
df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.pcd_dwelling_ownership")

In [174]:
#Writing to database
types = ['usual_residents', 'nonusual_residents']
df_pandas = pd.DataFrame(data = types, columns = ["name"])
df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.pcd_resident_types")

In [175]:
#Writing to database
types = ['house_detached', 'house_semidetached', 'house_terraced', 'flat']
df_pandas = pd.DataFrame(data = types, columns = ["name"])
df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.pcd_dwelling_types")

In [176]:
#Writing to database
types = ['economic_inactive', 'employee', 'self-employed', 'unemployed', 'student']
df_pandas = pd.DataFrame(data = types, columns = ["name"])
df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.pcd_economic_activity")

In [177]:
#Writing to database
types = ['unqualified', 'level_1', 'level_2', 'apprenticeship', 'level_3', 'level_4', 'other']
df_pandas = pd.DataFrame(data = types, columns = ["name"])
df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.pcd_education_level")

In [178]:
#Writing to database
types = ['good_health', 'fair_health', 'bad_health']
df_pandas = pd.DataFrame(data = types, columns = ["name"])
df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.pcd_health_status")

In [179]:
#Writing to database
foreign_key_map = read_db("rental_schema.postcodes").toPandas()
foreign_key_map = foreign_key_map[["id", "postcode"]]
foreign_key_map = {postcode: value for value, postcode in zip(foreign_key_map["id"], foreign_key_map["postcode"])}

df_pandas = df[["address", "postcode"]].drop_duplicates()
df_pandas["postcode"] = df_pandas["postcode"].map(foreign_key_map)
df_pandas.columns = ["full_address", "postcode_id"]

df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.address")

In [180]:
#Defining function to write many-to-many postcode to category share tables to database
def pcd_share_write_to_db(cat_table, df_columns, cat_id_column, share_table):
    foreign_key_map_pcd = read_db("rental_schema.postcodes").toPandas()
    foreign_key_map_pcd = foreign_key_map_pcd[["id", "postcode"]]
    foreign_key_map_pcd = {postcode: value for value, postcode in zip(foreign_key_map_pcd["id"], foreign_key_map_pcd["postcode"])}

    foreign_key_map_cat = read_db(cat_table).toPandas()
    foreign_key_map_cat = foreign_key_map_cat[["id", "name"]]
    foreign_key_map_cat = {name: value for value, name in zip(foreign_key_map_cat["id"], foreign_key_map_cat["name"])}

    df_pandas = df[df_columns].drop_duplicates()
    df_pandas = pd.melt(df_pandas, id_vars = df_columns[0], value_vars = df_columns[1:])
    df_pandas["postcode"] = df_pandas["postcode"].map(foreign_key_map_pcd)
    df_pandas["variable"] = df_pandas["variable"].map(foreign_key_map_cat)
    df_pandas.columns = ["postcode_id", cat_id_column, "share"]
    
    df_spark = spark.createDataFrame(df_pandas)
    write_to_db(df_spark, share_table)

In [181]:
#Writing to database
pcd_share_write_to_db("rental_schema.pcd_age_bins", 
                      ['postcode', 'ages_0_15', 'ages_16_29', 'ages_30_44', 'ages_45_65', 'ages_66_plus'], 
                      "age_bin_id", 
                      "rental_schema.pcd_age_bin_share")

In [182]:
#Writing to database
pcd_share_write_to_db("rental_schema.pcd_dwelling_ownership", 
                      ['postcode', 'couple_children_dependent', 'couple_children_nondependent', 'lone_parent', 'single', 'other_multi'], 
                      "dwelling_ownership_id", 
                      "rental_schema.pcd_dwelling_ownership_share")

In [183]:
#Writing to database
pcd_share_write_to_db("rental_schema.pcd_household_types", 
                      ['postcode', 'owned_outright', 'owned_mortgage', 'rented_social', 'rented_private'], 
                      "household_type_id", 
                      "rental_schema.pcd_household_type_share")

In [184]:
#Writing to database
pcd_share_write_to_db("rental_schema.pcd_resident_types", 
                      ['postcode', 'usual_residents', 'nonusual_residents'], 
                      "resident_type_id", 
                      "rental_schema.pcd_resident_type_share")

In [185]:
#Writing to database
pcd_share_write_to_db("rental_schema.pcd_dwelling_types", 
                      ['postcode', 'house_detached', 'house_semidetached', 'house_terraced', 'flat'], 
                      "dwelling_type_id", 
                      "rental_schema.pcd_dwelling_type_share")

In [186]:
#Writing to database
pcd_share_write_to_db("rental_schema.pcd_economic_activity", 
                      ['postcode', 'economic_inactive', 'employee', 'self-employed', 'unemployed', 'student'], 
                      "economic_activity_id", 
                      "rental_schema.pcd_economic_activity_share")

In [187]:
#Writing to database
pcd_share_write_to_db("rental_schema.pcd_education_level", 
                      ['postcode', 'unqualified', 'level_1', 'level_2', 'apprenticeship', 'level_3', 'level_4', 'other'], 
                      "education_level_id", 
                      "rental_schema.pcd_education_level_share")

In [188]:
#Writing to database
pcd_share_write_to_db("rental_schema.pcd_health_status", 
                      ['postcode', 'good_health', 'fair_health', 'bad_health'], 
                      "health_status_id", 
                      "rental_schema.pcd_health_status_share")

In [201]:
#Writing to database
foreign_key_map_source = read_db("rental_schema.sources").toPandas()
foreign_key_map_source = foreign_key_map_source[["id", "name"]]
foreign_key_map_source = {name: value for value, name in zip(foreign_key_map_source["id"], foreign_key_map_source["name"])}

foreign_key_map_address = read_db("rental_schema.address").toPandas()
foreign_key_map_address = foreign_key_map_address[["id", "full_address"]]
foreign_key_map_address = {name: value for value, name in zip(foreign_key_map_address["id"], foreign_key_map_address["full_address"])}

foreign_key_map_type = read_db("rental_schema.types").toPandas()
foreign_key_map_type = foreign_key_map_type[["id", "name"]]
foreign_key_map_type = {name: value for value, name in zip(foreign_key_map_type["id"], foreign_key_map_type["name"])}

df_pandas = df[["bedrooms", "bathrooms", "nearest_station", "price", "size_imputed", "type", "address", "source"]]
df_pandas["source"] = df_pandas["source"].map(foreign_key_map_source)
df_pandas["address"] = df_pandas["address"].map(foreign_key_map_address)
df_pandas["type"] = df_pandas["type"].map(foreign_key_map_type)
df_pandas.columns = ["bedrooms", "bathrooms", "nearest_station_distance", "price", "size_imputed", "type_id", "address_id", "source_id"]

df_spark = spark.createDataFrame(df_pandas)
write_to_db(df_spark, "rental_schema.listings")

<ipython-input-201-e1867fb84c8a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pandas["source"] = df_pandas["source"].map(foreign_key_map_source)
<ipython-input-201-e1867fb84c8a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pandas["address"] = df_pandas["address"].map(foreign_key_map_address)
<ipython-input-201-e1867fb84c8a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i